# VBA Files

### Pulling Excel Files from the Website

In [1]:
# There are weekly files from 2014 to present on https://www.benefits.va.gov/reports/detailed_claims_data.asp

# There are historical files at https://www.benefits.va.gov/REPORTS/mmwr/historical/{year}/index.asp from 2004 to 2013

# Presumably the first step will be just pulling all of them

### Libraries/Setup

In [2]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
os.chdir(r'C:\Users\admin\Documents\Python Scripts\veteranLawData')

### Get File URLs

In [3]:
def pullTheFiles(year):
    url= f'https://www.benefits.va.gov/REPORTS/mmwr/historical/{str(year)}/index.asp'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    filesToGet=[]
    for link in soup.findAll('a'):
        try:
            if "xls" in (link.get('href')):
                filesToGet.append(link.get('href'))
        except:
            pass
    return(filesToGet)

yearRange=list(range(2004, 2014))
#files=pullTheFiles(2004)
filesList=list(map(pullTheFiles, yearRange))
flatFileList = [item for sublist in filesList for item in sublist]

### Check We Have The Right Number: 2008 Is Legitimately Missing Three, I Checked

In [4]:
for fileList in filesList:
    print(len(fileList))

52
52
52
52
49
52
52
52
52
53


### Pull The Files

In [4]:
def pullFile(url):
    cleanLink='https://www.benefits.va.gov'+url
    r = requests.get(cleanLink, allow_redirects=True)
    name=cleanLink.split("/")[-1]
    open(name, 'wb').write(r.content)
    return(name)

#listOfFileNames=list(map(pullFile, files))
filesNameList=list(map(pullFile, flatFileList))

### Read Them Into A Dictionary

In [5]:
def readBasicFile(fileName):
    try:
        df=pd.read_excel(fileName)
        return(df)
    except:
        print(f"failed on {fileName}")

listOfDFs=list(map(readBasicFile, filesNameList))
dictionaryOfFiles = dict(zip(filesNameList, listOfDFs))

failed on 102113.xls
failed on 102813.xls
failed on 110413.xls
failed on 111213.xls
failed on 111813.xls
failed on 112513.xls
failed on 120213.xls
failed on 120913.xls
failed on 121613.xls
failed on 122313.xls
failed on 123013.xls


### Append Into Master DF

In [6]:
masterDF=pd.DataFrame()
for key in dictionaryOfFiles.keys():
    try:
        df=dictionaryOfFiles[key]
        df['fileName']=key
        listOfPhrasesToFind=["As of", "For the Week"]
        foundPhrase=[i for i in df.columns if any(substring in i for substring in listOfPhrasesToFind)]
        dropPhrase=[i for i in df.columns if "Percent" in i]
        #print(len(foundPhrase))
        df["forTheWeek"]=df[foundPhrase[0]]
        df=df.drop([foundPhrase[0]], axis=1)
        df=df.drop(dropPhrase, axis=1)
        masterDF=masterDF.append(df, sort=True)
    except:
        print(f'failed on {key}')

failed on NewMMWL_100509.xls
failed on 101309.xls
failed on 101909.xls
failed on 102609.xls
failed on 110209.xls
failed on 110909.xls
failed on 111609.xls
failed on 112309.xls
failed on 113009.xls
failed on 120709.xls
failed on 121409.xls
failed on 122109.xls
failed on 122809.xls
failed on 010410.xlsx
failed on 040510.xlsx
failed on 070610.xlsx
failed on 100410.xlsx
failed on 011110.xlsx
failed on 041210.xlsx
failed on 071210.xlsx
failed on 101210.xlsx
failed on 011910.xlsx
failed on 041910.xlsx
failed on 071910.xlsx
failed on 101810.xlsx
failed on 012510.xlsx
failed on 042610.xlsx
failed on 072610.xlsx
failed on 102510.xlsx
failed on 020110.xlsx
failed on 050310.xlsx
failed on 080210.xlsx
failed on 110110.xlsx
failed on 020810.xlsx
failed on 051010.xlsx
failed on 080910.xlsx
failed on 110810.xlsx
failed on 021610.xlsx
failed on 051710.xlsx
failed on 081610.xlsx
failed on 111510.xlsx
failed on 022210.xlsx
failed on 052410.xlsx
failed on 082310.xlsx
failed on 112210.xlsx
failed on 03011

In [7]:
masterDF.columns

Index([' ', ' Rating Cases Pending', ' Rating Claims Pending',
       '* Scorecard  Non-Rating Cases Pending',
       '* Scorecard  Non-Rating Claims Pending', 'C&P Claims over 180 Days ',
       'C&P Claims over 180 Days in WIPP', 'C&P Work Items over 180 Days ',
       'COE', 'COEs Issued', 'Date of Oldest Pending COE',
       'Education Cases Pending ', 'Education Work Items Pending ',
       'Guarantees Pending', 'IVMs Pending (EP 154 and EP 314)',
       'Non-Rating Cases Pending over 180 Days',
       'Non-Rating Claims Pending over 180 Days', 'Pre-Discharge Claims',
       'Rating Cases Pending', 'Rating Cases Pending over 180 Days',
       'Rating Claims Pending over 180 Days', 'SOC's',
       'Scorecard Non-Rating Cases Pending', 'Scorecard Rating Cases Pending',
       'Total Appeals Requiring Adjudicative Action ',
       'Total Appeals Requiring Adjudicative action ', 'Total C&P Pending ',
       'Total C&P Pending in Wipp', 'Total C&P Work Items Pending ',
       'Unnamed:

### Right now: it looks like 04-09 is working; after that we need to look at the format of the 09 worksheets to modify it; there's also some column cleaning we may need to do for the 04-09 group; this is a good stopping point

### Are we missing lines? spot checked first and last, looked good, getting weird value counts for the regional variable, but the overall counts don't have too much variation

In [23]:
seriesOfLength=pd.Series()

for file in masterDF['fileName'].unique():
    s= pd.Series([len(masterDF.loc[masterDF['fileName']==file])], index=[file])
    seriesOfLength=seriesOfLength.append(s)

In [24]:
seriesOfLength.value_counts()

71    84
72    83
70    78
68    24
78    15
73    11
dtype: int64

In [25]:
seriesOfLength.loc[seriesOfLength==68]

010504.xls    68
040504.xls    68
012004.xls    68
041204.xls    68
041904.xls    68
012604.xls    68
042604.xls    68
020204.xls    68
050304.xls    68
020904.xls    68
051004.xls    68
021704.xls    68
051704.xls    68
022304.xls    68
052404.xls    68
030104.xls    68
060104.xls    68
030804.xls    68
060704.xls    68
031504.xls    68
061404.xls    68
032204.xls    68
062104.xls    68
032904.xls    68
dtype: int64

### Looking at One With Count 68 : Looks Good

In [27]:
masterDF.loc[masterDF['fileName']=='021704.xls']

,,Rating Cases Pending,Rating Claims Pending,* Scorecard Non-Rating Cases Pending,* Scorecard Non-Rating Claims Pending,C&P Claims over 180 Days,C&P Claims over 180 Days in WIPP,C&P Work Items over 180 Days,COE,COEs Issued,...,Total Appeals Requiring Adjudicative Action,Total Appeals Requiring Adjudicative action,Total C&P Pending,Total C&P Pending in Wipp,Total C&P Work Items Pending,Unnamed: 18,VACOLS Appeals,VACOLS Appeals includes AMC & Travel Board,fileName,forTheWeek
0,NaT,NaN,NaN,NaN,NaN,NaN,102909.000000,NaN,NaT,1906-11-01 00:00:00,...,NaN,124637.000000,NaN,478130.000000,NaN,NaN,NaN,145668.000000,021704.xls,USA
1,NaT,NaN,NaN,NaN,NaN,NaN,104386.000000,NaN,NaT,1906-11-01 00:00:00,...,NaN,125006.000000,NaN,470643.000000,NaN,NaN,NaN,145132.000000,021704.xls,Last Week
2,NaT,NaN,NaN,NaN,NaN,NaN,-1477.000000,NaN,NaT,00:00:00,...,NaN,-369.000000,NaN,7487.000000,NaN,NaN,NaN,536.000000,021704.xls,Change from Last Week
3,NaT,NaN,NaN,NaN,NaN,NaN,-0.014149,NaN,NaT,NaN,...,NaN,-0.002952,NaN,0.015908,NaN,NaN,NaN,0.003693,021704.xls,Percent Change
4,NaT,NaN,NaN,NaN,NaN,NaN,118461.000000,NaN,NaT,1906-11-01 00:00:00,...,NaN,97678.000000,NaN,472859.000000,NaN,NaN,NaN,121596.000000,021704.xls,Last Year
5,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,021704.xls,EASTERN AREA
6,NaT,NaN,NaN,NaN,NaN,NaN,1890.000000,NaN,NaT,NaN,...,NaN,1165.000000,NaN,6161.000000,NaN,NaN,NaN,1466.000000,021704.xls,Baltimore Regional Office
7,NaT,NaN,NaN,NaN,NaN,NaN,2757.000000,NaN,NaT,NaN,...,NaN,1568.000000,NaN,7641.000000,NaN,NaN,NaN,1806.000000,021704.xls,Boston Regional Office
8,NaT,NaN,NaN,NaN,NaN,NaN,2147.000000,NaN,NaT,NaN,...,NaN,2040.000000,NaN,6959.000000,NaN,NaN,NaN,2319.000000,021704.xls,Buffalo Regional Office
9,NaT,NaN,NaN,NaN,NaN,NaN,3610.000000,NaN,NaT,NaN,...,NaN,3110.000000,NaN,12994.000000,NaN,NaN,NaN,3960.000000,021704.xls,Cleveland Regional Office


### Now Let's Look at the Next Template // first failure to read in this way

#### The first failure is 101309.xls; looking at it, there are two tabs, the second one is called "Final Aggregate" and it also differs from the previous ones in that there's more spacing/headers

In [30]:
fileName="101309.xls"
df=pd.read_excel(fileName, sheet_name='Final Aggregate', header=7)

In [31]:
df

Claims Pending  Pending over 125 days  \
NaN USA                               393195                 144017   
    EASTERN AREA                       83069                  33642   
          Baltimore                     8647                   4387   
          Boston                        4274                   1203   
          Buffalo                       5172                   2324   
          Cleveland                    10208                   4055   
          Detroit                      11541                   5577   
          Hartford                      1561                    404   
          Indianapolis                 10353                   4714   
          Manchester                    1264                    409   
          New York                      7187                   3341   
          Newark                        3379                   1149   
          Philadelphia                 10274                   2764   
          Pittsburgh                    4679                   2151   
          Providence                    1526                    358   
          Togus                         1772                    427   
          White River J.                 721                    275   
          Wilmington                     511                    104   
    SOUTHERN AREA                     125437                  47222   
          Atlanta                      18379                   7683   
          Columbia                      9112                   2930   
          Huntington                    2063                    351   
          Jackson                       5945                   2942   
          Louisville                    4821                   1354   
          Montgomery                   11478                   5702   
          Nashville                     7293                   1639   
          Roanoke                      13125                   4827   
          San Juan                      3528                   1391   
          St. Petersburg               27534                  11069   
          Washington                      15                      8   
...                                      ...                    ...   
          St. Paul                      8617                   1418   
          Waco                         18758                   6851   
          Wichita                       2759                    802   
    WESTERN AREA                       83574                  30363   
          Albuquerque                   2945                    976   
          Anchorage                     1408                    637   
          Boise                         1255                    208   
          Denver                        6458                   2099   
          Fort Harrison                 1611                    403   
          Honolulu                      2541                    971   
          Los Angeles                   8054                   2790   
          Manila                        1427                    381   
          Oakland                      15199                   7266   
          Phoenix                       7928                   2652   
          Portland                      6042                   2344   
          Reno                          4222                   1750   
          Salt Lake City                5159                   1535   
          San Diego                     9869                   3115   
          Seattle                       9456                   3236   
    Other                                  1                      0   
    NaN                                  NaN                    NaN   
    NaN                    PENSION INVENTORY                    NaN   
    NaN                       Entitlement                       NaN   
    NaN                       Claims Pending  Pending over 125 days   
    NaN                        120, 180, 190          

In [ ]:
### This is working fine, the question is, can we actually hardcode 7 or do we need to make this more flexible?